In [ ]:
!pip install openpyxl
!pip install python-docx

In [ ]:
import pandas as pd
import numpy as np
import warnings
import random
warnings.filterwarnings("ignore")

available_players = ["nsilva", "spinto", "apimenta", "rpinheiro", "ggomes",
                     "guest_random", "guest_jlopes", "pduarte", "guest_cgomes", "jsilva"]

anygoal_data = pd.read_csv("anygoal_season_2022_resume.csv")

anybrain_df = pd.read_excel("players_01_22.xlsx")
anybrain_df.drop([7], axis=0, inplace=True)
anybrain_df.set_index('Unnamed: 0', inplace=True)
anybrain_df = anybrain_df.T.reset_index()
anybrain_df.columns = ['Nome', 'Ataque', 'Tecnica', 'Tatica', 'Defesa', 'Criatividade', 
                       'Fisico', 'GK', 'Overall', 'Overall_No_GK']
anybrain_df.index.name = None
anybrain_df.Nome.replace({'Jpab' : 'guest_jpab', 'J lopes' : 'guest_jlopes', 'carlos' : 'guest_cgomes', 
                          'Mineiro' : 'nsilva', 'Nelo' : 'esousa', 'Ze' : 'jsilva', 'Fintas' : 'spinto', 
                          'Pimenta' : 'apimenta', 'Xerife' : 'rpinheiro', 'Champion' : 'pduarte', 
                          'Gus' : 'ggomes', 'Costeira' : 'gcosteira', 'Flash' : 'guest_flash', 'random' : 'guest_random'}, inplace=True)

# Merge the anybrain_df with anygoal
merged_df = anybrain_df.merge(anygoal_data, left_on='Nome', right_on='Player')
merged_df.drop(['Unnamed: 0', 'Player', 'MVP_%', 'Victories', 'Losses', 'Victories_%', 
                'Goals/Games', 'Team_Goals', 'Team_Suffered', 'Team_Influence', 'Team_Goals/Games', 
                'Team_Suffered/Games', 'Biggest_W_Streak', 'Biggest_L_Streak'], axis=1, inplace=True)

player_stats = ['Ataque', 'Tecnica', 'Tatica', 'Defesa', 'Criatividade',
                'Fisico', 'GK', 'Overall', 'Overall_No_GK', 'Point_System']

In [ ]:
data_players = merged_df.loc[merged_df.Nome.isin(available_players), :]
data_players

# First Version

In [ ]:
def get_stats(df):
    return df.mean(), df.median()

def split_df(df):
    half_length = df.shape[0] // 2
    df1 = df.iloc[:half_length, :]
    df2 = df.iloc[half_length:, :]
    return df1, df2

def teams_conditions(mean_1, mean_2, median_1, median_2):   
    mean_1_2 = mean_1.subtract(mean_2).abs()
    median_1_2 = median_1.subtract(median_2).abs()
    
    mean_condition = [(mean_1_2.Point_System < 1.1)]+[(mean_1_2[x]< 0.69) for x in player_stats]
    median_condition = [(median_1_2.Point_System < 1.1)]+[(median_1_2[x]< 0.69) for x in player_stats]
    
    return (all(mean_condition) or all(median_condition))

def balance_dfs(df):
    df1, df2 = split_df(df)
    
    df_mean1, df_median1 = get_stats(df1)
    df_mean2, df_median2 = get_stats(df2)
    counter = 20000
    while (not teams_conditions(df_mean1, df_mean2, df_median1, df_median2)):
        i1 = np.random.randint(df1.shape[0])
        i2 = np.random.randint(df2.shape[0])
        row1, row2 = df1.iloc[i1, :], df2.iloc[i2, :]
        df1.iloc[i1, :] = row2.values
        df2.iloc[i2, :] = row1.values
        df_mean1, df_median1 = get_stats(df1)
        df_mean2, df_median2 = get_stats(df2)
        
        if counter > 0:
            counter -= 1
        else:
            print("After 20k combinations, i couldn't satisfy the conditions")
            break
            
    return df1, df2

In [ ]:
df1, df2 = balance_dfs(data_players)

In [ ]:
print(df1.Nome.values)
df1.describe()

In [ ]:
print(df2.Nome.values)
df2.describe()

In [ ]:
tmp = ["Jogador_1", "Jogador_2", "Jogador_3", "Jogador_4", "Jogador_5", "Jogador_6", "Jogador_7", "Jogador_8", "Jogador_9", "Jogador_10"]
tmp2 = df2.Nome.values.tolist() + df1.Nome.values.tolist()

full_players = dict(zip(tmp,tmp2))


In [ ]:
import os 

# Meti isto a apagar o ficheiro caso já existe, porque se existir, isto vai dar um erro qualquer de permissões 

if os.path.exists("./Anygoal_Template.docx"):
    os.remove("./Anygoal_Template.docx")

In [ ]:
import docx
from docx.shared import Pt


doc = docx.Document("./template/Anygoal_Template.docx")


for table in doc.tables:
    for row in table.rows:
        for cell in row.cells:
            if cell.text in full_players:
                cell.text = cell.text.replace(cell.text, full_players[cell.text])


doc.save("./Anygoal_Template.docx")
